In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, LSTM, GRU, GlobalAveragePooling1D, Lambda, Dropout, Concatenate, GlobalMaxPooling1D, MaxPool1D, AveragePooling1D
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder

In [2]:
# Clear the TensorFlow graph
tf.compat.v1.reset_default_graph()

# Clear the session
tf.keras.backend.clear_session()

In [3]:
sales = pd.read_csv('./train.csv')
oil = pd.read_csv('./oil.csv')
events = pd.read_csv('./holidays_events.csv')

missing_dates = ['2013-12-26', '2014-12-26', '2015-12-26', '2016-12-26']
missing_dates = pd.to_datetime(missing_dates, format='%Y-%m-%d')

events['date'] = pd.to_datetime(events['date'], format='%Y-%m-%d')
events = events.loc[(events['date'] >= '2013-01-01')
                     & (events['date'] <= '2017-08-15')]
events = events.drop_duplicates(subset='date', keep='last')
events = events.set_index('date')
events = events.resample('D').asfreq().fillna({'type':'Work Day', 'locale':'National', 'locale_name':'None', 'description':'Not a holiday', 'transferred':True})
for i, _ in events.iterrows():
  if(i in missing_dates):
    events.drop(index=[i], inplace=True)

oil_prices = []
for price in oil['dcoilwtico'].to_numpy():
  if(np.isnan(price)):
    continue
  oil_prices.append(price)
oil['date'] = pd.to_datetime(oil['date'])
oil = oil.loc[(oil['date'] >= '2013-01-01')
                     & (oil['date'] <= '2017-08-15')]
oil = oil.drop_duplicates(subset='date', keep='last')
oil = oil.set_index('date')
oil = oil.resample('D').asfreq().fillna(np.mean(oil_prices))
for i, _ in oil.iterrows():
  if(i in missing_dates):
    oil.drop(index=[i], inplace=True)

sales = sales.drop('onpromotion', axis=1)
sales['date'] = pd.to_datetime(sales['date'])
sales.loc[sales['family'] == 'BREAD/BAKERY', 'family'] = 'BAKERY'

In [4]:
# Checking if there are missing days in the sales data
dates = sales[sales['family']=='MAGAZINES'][sales['store_nbr'] == 54]['date'].to_numpy();
for i, date in enumerate(dates):
  if(dates[int(i)] - dates[int(i)-1] > np.timedelta64(86400000000000,'ns')):
    print(f'Day {dates[int(i)]} was skipped');

Day 2013-12-26T00:00:00.000000000 was skipped
Day 2014-12-26T00:00:00.000000000 was skipped
Day 2015-12-26T00:00:00.000000000 was skipped
Day 2016-12-26T00:00:00.000000000 was skipped


C:\Users\Nikita\AppData\Local\Temp\ipykernel_13392\1458558262.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dates = sales[sales['family']=='MAGAZINES'][sales['store_nbr'] == 54]['date'].to_numpy();


In [5]:
sales_families = sales['family'].unique()
sales_families

array(['AUTOMOTIVE', 'BABY CARE', 'BEAUTY', 'BEVERAGES', 'BOOKS',
       'BAKERY', 'CELEBRATION', 'CLEANING', 'DAIRY', 'DELI', 'EGGS',
       'FROZEN FOODS', 'GROCERY I', 'GROCERY II', 'HARDWARE',
       'HOME AND KITCHEN I', 'HOME AND KITCHEN II', 'HOME APPLIANCES',
       'HOME CARE', 'LADIESWEAR', 'LAWN AND GARDEN', 'LINGERIE',
       'LIQUOR,WINE,BEER', 'MAGAZINES', 'MEATS', 'PERSONAL CARE',
       'PET SUPPLIES', 'PLAYERS AND ELECTRONICS', 'POULTRY',
       'PREPARED FOODS', 'PRODUCE', 'SCHOOL AND OFFICE SUPPLIES',
       'SEAFOOD'], dtype=object)

In [6]:
sales_stores = np.sort(sales[sales['family']=='BEVERAGES']['store_nbr'].unique())
sales_stores

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54], dtype=int64)

In [7]:
sales

,id,date,store_nbr,family,sales
0,0,2013-01-01,1,AUTOMOTIVE,0.000
1,1,2013-01-01,1,BABY CARE,0.000
2,2,2013-01-01,1,BEAUTY,0.000
3,3,2013-01-01,1,BEVERAGES,0.000
4,4,2013-01-01,1,BOOKS,0.000
...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553
3000885,3000885,2017-08-15,9,PRODUCE,2419.729
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000


In [8]:
sales[sales['family']=='BEVERAGES'].count()

id           90936
date         90936
store_nbr    90936
family       90936
sales        90936
dtype: int64

In [9]:
sales['family'].unique()

array(['AUTOMOTIVE', 'BABY CARE', 'BEAUTY', 'BEVERAGES', 'BOOKS',
       'BAKERY', 'CELEBRATION', 'CLEANING', 'DAIRY', 'DELI', 'EGGS',
       'FROZEN FOODS', 'GROCERY I', 'GROCERY II', 'HARDWARE',
       'HOME AND KITCHEN I', 'HOME AND KITCHEN II', 'HOME APPLIANCES',
       'HOME CARE', 'LADIESWEAR', 'LAWN AND GARDEN', 'LINGERIE',
       'LIQUOR,WINE,BEER', 'MAGAZINES', 'MEATS', 'PERSONAL CARE',
       'PET SUPPLIES', 'PLAYERS AND ELECTRONICS', 'POULTRY',
       'PREPARED FOODS', 'PRODUCE', 'SCHOOL AND OFFICE SUPPLIES',
       'SEAFOOD'], dtype=object)

In [10]:
oil

,dcoilwtico
date,
2013-01-01,67.714366
2013-01-02,93.140000
2013-01-03,92.970000
2013-01-04,93.120000
2013-01-05,67.714366
...,...
2017-08-11,48.810000
2017-08-12,67.714366
2017-08-13,67.714366


In [11]:
events

,type,locale,locale_name,description,transferred
date,,,,,
2013-01-01,Holiday,National,Ecuador,Primer dia del ano,False
2013-01-02,Work Day,National,None,Not a holiday,True
2013-01-03,Work Day,National,None,Not a holiday,True
2013-01-04,Work Day,National,None,Not a holiday,True
2013-01-05,Work Day,National,Ecuador,Recupero puente Navidad,False
...,...,...,...,...,...
2017-08-11,Transfer,National,Ecuador,Traslado Primer Grito de Independencia,False
2017-08-12,Work Day,National,None,Not a holiday,True
2017-08-13,Work Day,National,None,Not a holiday,True


In [13]:
# Creating a function to draw graphs of each store sales for a specified product
def draw_sale_graphs_for_family(family_name, store_amount):
  for store in sales_stores[:store_amount]:
    sales_numpy = sales[sales['family'] == family_name][sales['store_nbr'] == store]['sales'].to_numpy()
    fig, ax = plt.subplots(figsize=(10,7))
    ax.bar(range(len(sales_numpy)), sales_numpy)
    ax.set(title=f'Store number: {str(store)}');

In [14]:
# draw_sale_graphs_for_family('BEVERAGES', len(sales_stores))

In [15]:
# Preparing data, converting to numbers
beverage_1_sales = sales[sales['family']=='BEVERAGES'][sales['store_nbr'] == 1]['sales'].to_numpy()
oil_arr = oil['dcoilwtico'].to_numpy()
onehot = OneHotEncoder()
events_arr = onehot.fit_transform(events).todense()

C:\Users\Nikita\AppData\Local\Temp\ipykernel_13392\2502400928.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  beverage_1_sales = sales[sales['family']=='BEVERAGES'][sales['store_nbr'] == 1]['sales'].to_numpy()


In [16]:
def preprocess_sales(sales, oil, events):
  label = sales[-1]
  target = sales[:-1]
  return (target, oil, events), label

In [17]:
def create_dataset(sales_arr, oil_arr, events_arr, window_size):
  sales_data = tf.data.Dataset.from_tensor_slices(sales_arr).batch(window_size+1, drop_remainder=True)
  oil_data = tf.data.Dataset.from_tensor_slices(oil_arr).batch(window_size, drop_remainder=True)
  events_data = tf.data.Dataset.from_tensor_slices(events_arr).batch(window_size, drop_remainder=True)

  train_data = tf.data.Dataset.zip((sales_data, oil_data, events_data))
  train_dataset = train_data.map(preprocess_sales).batch(32, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
  return train_dataset

In [18]:
beverage_1_dataset = create_dataset(beverage_1_sales, oil_arr, events_arr, window_size=7)

In [79]:
for targets, labels in beverage_1_dataset.take(1):
  print(f"Sales data: {targets[0][0]}")
  print(f"Oil data: {targets[1][0]}")
  print(f"Events data: {targets[2][0].shape}")
  print(f"Predicted label: {labels[0]}")

Sales data: [   0. 1091.  919.  953. 1160.  407. 1158.]
Oil data: [67.71436596 93.14       92.97       93.12       67.71436596 67.71436596
 93.2       ]
Events data: (7, 132)
Predicted label: 1029.0


In [20]:
def create_model_callbacks(epochs, monitor='loss', filepath='/checkpoints/'):
  model_checkpoint = tf.keras.callbacks.ModelCheckpoint(monitor=monitor, save_best_only=True, filepath=filepath, save_weights_only=True, verbose=0)
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor=monitor, patience=int(epochs/5))
  reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor=monitor, patience=int(epochs/10))
  return [model_checkpoint, early_stopping, reduce_lr]

In [21]:
WINDOW_SIZE=7
N_NEURONS=1024
EPOCHS=250

In [ ]:
WINDOW_SIZE=7
N_NEURONS=1024
EPOCHS=250

sales_inputs = Input(shape=(WINDOW_SIZE), name='sales_input')
x = Dense(N_NEURONS, activation='relu')(sales_inputs)
x = Dense(N_NEURONS, activation='relu')(x)
sales_outputs = Dense(int(N_NEURONS/2), activation='relu')(x)
sales_model = tf.keras.Model(sales_inputs, sales_outputs)

oil_inputs = Input(shape=(WINDOW_SIZE))
y = Dense(N_NEURONS, activation='relu')(oil_inputs)
y = Dense(N_NEURONS, activation='relu')(y)
oil_outputs = Dense(int(N_NEURONS/2), activation='relu')(y)
oil_model = tf.keras.Model(oil_inputs, oil_outputs)

events_inputs = Input(shape=(WINDOW_SIZE, 132))
z = Dense(N_NEURONS, activation='relu', name='dense_events_1')(events_inputs)
z = Dense(int(N_NEURONS/2), activation='relu', name='dense_events_2')(z)
z = GlobalMaxPooling1D(name='global_average_pooling__events')(z)
events_outputs = Dense(N_NEURONS, activation='relu')(z)
events_model = tf.keras.Model(events_inputs, events_outputs)

sales_oil_events_concat = Concatenate(name='sales_oil_events_concat', axis=-1)([sales_model.output, oil_model.output, events_model.output])

a = Dense(N_NEURONS, activation='relu', name='dense_concat_1')(sales_oil_events_concat)
outputs = Dense(1, activation='relu', name='concat_output_dense')(a)

model = tf.keras.Model([sales_inputs, oil_inputs, events_inputs], outputs)

In [ ]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 7, 132)]     0           []                               
                                                                                                  
 sales_input (InputLayer)       [(None, 7)]          0           []                               
                                                                                                  
 input_1 (InputLayer)           [(None, 7)]          0           []                               
                                                                                                  
 dense_events_1 (Dense)         (None, 7, 1024)      136192      ['input_2[0][0]']                
                                                                                            

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.mean_absolute_error,
              metrics=['mae'])

In [ ]:
model.fit(beverage_1_dataset,
          epochs=EPOCHS,
          callbacks=create_model_callbacks(EPOCHS, monitor='loss', filepath=r'C:\Users\Nikita\Desktop\store-sales\checkpoints\checkpoint.h5'))

Epoch 1/250
6/6 [==============================] - 1s 15ms/step - loss: 1019.6604 - mae: 1019.6604 - lr: 0.0010
Epoch 2/250
6/6 [==============================] - 0s 17ms/step - loss: 548.3940 - mae: 548.3940 - lr: 0.0010
Epoch 3/250
6/6 [==============================] - 0s 17ms/step - loss: 466.7440 - mae: 466.7440 - lr: 0.0010
Epoch 4/250
6/6 [==============================] - 0s 17ms/step - loss: 390.5273 - mae: 390.5273 - lr: 0.0010
Epoch 5/250
6/6 [==============================] - 0s 16ms/step - loss: 303.1080 - mae: 303.1080 - lr: 0.0010
Epoch 6/250
6/6 [==============================] - 0s 6ms/step - loss: 310.0035 - mae: 310.0035 - lr: 0.0010
Epoch 7/250
6/6 [==============================] - 0s 6ms/step - loss: 358.6200 - mae: 358.6200 - lr: 0.0010
Epoch 8/250
6/6 [==============================] - 0s 19ms/step - loss: 278.0124 - mae: 278.0124 - lr: 0.0010
Epoch 9/250
6/6 [==============================] - 0s 16ms/step - loss: 264.3332 - mae: 264.3332 - lr: 0.0010
Epoch 10/2

In [22]:
def create_sales_model(dataset, weights_ckp, WINDOW_SIZE=7, N_NEURONS=512, EPOCHS=250, ckp_name='checkpoint'):
  sales_inputs = Input(shape=(WINDOW_SIZE), name='sales_input')
  x = Dense(N_NEURONS, activation='relu')(sales_inputs)
  x = Dense(N_NEURONS, activation='relu')(x)
  sales_outputs = Dense(int(N_NEURONS/2), activation='relu')(x)
  sales_model = tf.keras.Model(sales_inputs, sales_outputs)

  oil_inputs = Input(shape=(WINDOW_SIZE))
  y = Dense(N_NEURONS, activation='relu')(oil_inputs)
  y = Dense(N_NEURONS, activation='relu')(y)
  oil_outputs = Dense(int(N_NEURONS/2), activation='relu')(y)
  oil_model = tf.keras.Model(oil_inputs, oil_outputs)

  events_inputs = Input(shape=(WINDOW_SIZE, 132))
  z = Dense(N_NEURONS, activation='relu', name='dense_events_1')(events_inputs)
  z = Dense(int(N_NEURONS/2), activation='relu', name='dense_events_2')(z)
  z = GlobalMaxPooling1D(name='global_average_pooling__events')(z)
  events_outputs = Dense(N_NEURONS, activation='relu')(z)
  events_model = tf.keras.Model(events_inputs, events_outputs)

  sales_oil_events_concat = Concatenate(name='sales_oil_events_concat', axis=-1)([sales_model.output, oil_model.output, events_model.output])

  a = Dense(N_NEURONS, activation='relu', name='dense_concat_1')(sales_oil_events_concat)
  outputs = Dense(1, activation='relu', name='concat_output_dense')(a)

  model = tf.keras.Model([sales_inputs, oil_inputs, events_inputs], outputs)

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.mean_absolute_error,
              metrics=['mae'])
  if(weights_ckp == False):
    model.fit(dataset,
          epochs=EPOCHS,
          verbose=0,
          callbacks=create_model_callbacks(EPOCHS, monitor='loss', filepath=f"./checkpoints/{ckp_name}.h5"))
  elif(type(weights_ckp) == 'str'):
    print(f'loading weights of: {weights_ckp}')
    model.load_weights(weights_ckp)
  return model

In [23]:
oil_arr = oil['dcoilwtico'].to_numpy()
onehot = OneHotEncoder()
events_arr = onehot.fit_transform(events).todense()

In [24]:
import os 
ckps = []
for ckp in os.listdir('./checkpoints'):
  ckps.append(ckp.split('.h5')[0])

In [25]:
total_stores = []
for family in sales['family'].unique():
  for store in sales[sales['family'] == family]['store_nbr'].unique():
    total_stores.append(store)
len(total_stores)

1782

In [26]:
len(ckps)

1783

In [ ]:
models = []
for family in sales['family'].unique():
  for store in sales[sales['family'] == family]['store_nbr'].unique():
    if(f"{family}-{store}" not in ckps):
      family_store_sales = sales[sales['family'] == family][sales['store_nbr'] == store]['sales'].to_numpy()
      dataset = create_dataset(family_store_sales, oil_arr, events_arr, window_size=7)
      model = create_sales_model(dataset, False, EPOCHS=100, ckp_name=f'{family}-{store}')
      models.append({f"{family}-{store}-model":model})

In [27]:
models_loaded = []
for family in sales['family'].unique():
  for store in sales[sales['family'] == family]['store_nbr'].unique():
    if(f"{family}-{store}" in ckps):
      if(len(models_loaded) < 892):
        family_store_sales = sales[sales['family'] == family][sales['store_nbr'] == store]['sales'].to_numpy()
        dataset = create_dataset(family_store_sales, oil_arr, events_arr, window_size=7)
        model = create_sales_model(dataset, weights_ckp=f"./checkpoints/{family}-{store}.h5", EPOCHS=1, ckp_name=f'{family}-{store}')
        models_loaded.append({f"{family}-{store}": model})

C:\Users\Nikita\AppData\Local\Temp\ipykernel_13392\222556851.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  family_store_sales = sales[sales['family'] == family][sales['store_nbr'] == store]['sales'].to_numpy()


In [28]:
len(models_loaded)

892

In [30]:
def map_preds_data(sales, oil, events):
  return (sales, oil, events)

In [38]:
def create_test_sets(sales, oil_arr, events_arr):
  preds_data = []
  for family in sales['family'].unique():
    for store in sales[sales['family'] == family]['store_nbr'].unique():
      if(len(preds_data) <= len(models_loaded)):
        sales_batch = tf.data.Dataset.from_tensor_slices(sales[sales['family'] == family][sales['store_nbr'] == store]['sales'].to_numpy()[-7:]).batch(7)
        oil_batch = tf.data.Dataset.from_tensor_slices(oil_arr[-7:]).batch(7)
        events_batch = tf.data.Dataset.from_tensor_slices(events_arr[-7:]).batch(7)
        preds_data_zip = tf.data.Dataset.zip((sales_batch, oil_batch, events_batch))
        preds_data_mapped = preds_data_zip.map(map_preds_data).batch(1).prefetch(tf.data.AUTOTUNE)
        preds_data.append(preds_data_mapped)
        # train_data = tf.data.Dataset.zip((sales_data, oil_data, events_data))
  return preds_data

In [39]:
preds_data = create_test_sets(sales, oil_arr, events_arr)

C:\Users\Nikita\AppData\Local\Temp\ipykernel_13392\4288163762.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  sales_batch = tf.data.Dataset.from_tensor_slices(sales[sales['family'] == family][sales['store_nbr'] == store]['sales'].to_numpy()[-7:]).batch(7)


In [40]:
for i in preds_data[0]:
  print(i)

(<tf.Tensor: shape=(1, 7), dtype=float64, numpy=array([[7., 9., 1., 6., 1., 1., 4.]])>, <tf.Tensor: shape=(1, 7), dtype=float64, numpy=
array([[49.59      , 48.54      , 48.81      , 67.71436596, 67.71436596,
        47.59      , 47.57      ]])>, <tf.Tensor: shape=(1, 7, 132), dtype=float64, numpy=
array([[[0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 

In [43]:
preds_data_loaded = preds_data[0:len(models_loaded)]

In [75]:
preds = []
for i, data in enumerate(preds_data_loaded):
  preds.append({f"{[name for name in models_loaded[i].keys()]}":[model.predict([target for target in data.take(1)]) for model in models_loaded[i].values()][0][0][0]})

In [78]:
preds

[{"['AUTOMOTIVE-1']": 0.0},
 {"['AUTOMOTIVE-10']": 0.0},
 {"['AUTOMOTIVE-11']": 0.0},
 {"['AUTOMOTIVE-12']": 1.0847895},
 {"['AUTOMOTIVE-13']": 0.55970883},
 {"['AUTOMOTIVE-14']": 2.4257631},
 {"['AUTOMOTIVE-15']": 0.0},
 {"['AUTOMOTIVE-16']": 0.9277293},
 {"['AUTOMOTIVE-17']": 1.3237827},
 {"['AUTOMOTIVE-18']": 0.0},
 {"['AUTOMOTIVE-19']": 0.0},
 {"['AUTOMOTIVE-2']": 0.0},
 {"['AUTOMOTIVE-20']": 0.0},
 {"['AUTOMOTIVE-21']": 0.844332},
 {"['AUTOMOTIVE-22']": 1.7605269},
 {"['AUTOMOTIVE-23']": 0.0},
 {"['AUTOMOTIVE-24']": 5.849148},
 {"['AUTOMOTIVE-25']": 0.6819629},
 {"['AUTOMOTIVE-26']": 0.0},
 {"['AUTOMOTIVE-27']": 0.700341},
 {"['AUTOMOTIVE-28']": 0.0},
 {"['AUTOMOTIVE-29']": 0.0},
 {"['AUTOMOTIVE-3']": 0.0},
 {"['AUTOMOTIVE-30']": 0.0},
 {"['AUTOMOTIVE-31']": 0.34245414},
 {"['AUTOMOTIVE-32']": 1.0336273},
 {"['AUTOMOTIVE-33']": 2.029542},
 {"['AUTOMOTIVE-34']": 0.0},
 {"['AUTOMOTIVE-35']": 2.925188},
 {"['AUTOMOTIVE-36']": 0.0},
 {"['AUTOMOTIVE-37']": 0.7871653},
 {"['AUTOMOTIVE-3